### Compilations

In [ ]:
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/cython_utils
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

### Dataset

In [18]:
# change train and test paths inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert_ann2dota.py

/workdir/msc/RotationDetection/cell_utils
100%|███████████████████████████████████████████| 30/30 [00:00<00:00, 71.89it/s]


In [21]:
# change root path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert2jpg.py

/workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
100%|█████████████████████████████████████████| 150/150 [00:02<00:00, 73.11it/s]


In [23]:
# change train path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python data_crop.py

/workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
class_list 2
find image 120
find label 120
0it [00:00, ?it/s]exp_L1.5_Luana_A172_cloneF9(mito)+H2B_TMZ3horas_100uM_2min_T1556616512
[]
1it [00:01,  1.52s/it]exp_LJ14_LuanaeJulieti_A172_nan_CT_10min_T1574821318
[]
2it [00:02,  1.49s/it]exp_LJ12.2_LuanaeJulieti_A172_F9DP53_TMZ3horas(conferir)_50uM_2min_T1558053744
[]
3it [00:04,  1.57s/it]exp_L1.2_Luana_A172_cloneF9(mito)+H2B_TMZ3horas_100uM_2min_T1554920443
[]
4it [00:06,  1.65s/it]exp_M1_Mauricio_mcf7_frame_10
[]
5it [00:08,  1.73s/it]exp_LJ1.4_LuanaeJulieti_A172(conferirnocaderno)_cloneF9(mito)(conferirnocaderno)_CT_30min_T1547953344
[]
6it [00:10,  1.68s/it]exp_L1_Luana_A172_cloneF9(mito)+H2B_TMZ3horas_100uM_15min_T1554255224
[]
7it [00:11,  1.60s/it]exp_M4_Mauricio_mcr5_frame_6
[]
8it [00:13,  1.68s/it]exp_LJ10_LuanaeJulieti_A172_cloneF9(mito)+H2B_CT_15min_T1553694354
[]
9it [00:14,  1.66s/it]exp_LJ12.2_LuanaeJulieti_A172_F9DP53_TMZ3horas(conferir)_50uM_2min_T1558020022
[]

In [37]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/  
!python convert_data_to_tfrecord.py --VOC_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/crop' \
                                   --xml_dir='labeltxt' \
                                   --image_dir='images' \
                                   --save_name='train'  \
                                   --img_format='.png'  \
                                   --save_dir='/workdir/datasets/msc/' \
                                   --dataset='UFRGS_CELL' \

/workdir/msc/RotationDetection/dataloader/dataset
/workdir/msc/RotationDetection

100%|██████████████████████████████████████| 5590/5590 [00:44<00:00, 126.16it/s]

Conversion is complete!


## Train

In [ ]:
%cd /workdir/msc/RotationDetection/tools/gwd
!python train.py

In [40]:
!git add . && git commit -m "updates to cell detector" && git push

## Test

In [ ]:
%cd /workdir/msc/RotationDetection/tools/gwd
!python test_hrsc2016.py --img_dir='/datasets/msc/HRSC2016/HRSC2016/Test/AllImages'  \
                         --gpu=0 \
                         --image_ext='.bmp' \
                         --test_annotation_path='/datasets/msc/HRSC2016/HRSC2016/Test/xmls'

In [ ]:
%cd /workdir/msc/RotationDetection/tools/gwd
!python test_dota.py --test_dir='/datasets/dataset/DOTA/test/images'  \
                     --gpus=0,1